# 获取数据

In [26]:
# 下载数据
import get_data

# 定义需要下载的数据
symbol = "ETHUSDT"          # 币种符号
interval = "15m"             # 时间间隔
start_date = "2019-12-31"   # 起始日期
end_date = "2024-09-08"     # 结束日期
data_type = "futures"          # 数据类型：现货 (spot) 或 合约 (futures)
data_folder = "data"        # 数据存储文件夹

# 调用下载函数
get_data.download_binance_data(
    symbol=symbol, 
    interval=interval, 
    start_date=start_date, 
    end_date=end_date, 
    data_type=data_type, 
    data_folder=data_folder
)

Downloaded: ETHUSDT-15m-2019-12-31.zip
Unzipped: downloads\ETHUSDT-15m-2019-12-31.zip
Deleted zip file: downloads\ETHUSDT-15m-2019-12-31.zip
Downloaded: ETHUSDT-15m-2020-01-01.zip
Unzipped: downloads\ETHUSDT-15m-2020-01-01.zip
Deleted zip file: downloads\ETHUSDT-15m-2020-01-01.zip
Downloaded: ETHUSDT-15m-2020-01-02.zip
Unzipped: downloads\ETHUSDT-15m-2020-01-02.zip
Deleted zip file: downloads\ETHUSDT-15m-2020-01-02.zip
Downloaded: ETHUSDT-15m-2020-01-03.zip
Unzipped: downloads\ETHUSDT-15m-2020-01-03.zip
Deleted zip file: downloads\ETHUSDT-15m-2020-01-03.zip
Downloaded: ETHUSDT-15m-2020-01-04.zip
Unzipped: downloads\ETHUSDT-15m-2020-01-04.zip
Deleted zip file: downloads\ETHUSDT-15m-2020-01-04.zip
Downloaded: ETHUSDT-15m-2020-01-05.zip
Unzipped: downloads\ETHUSDT-15m-2020-01-05.zip
Deleted zip file: downloads\ETHUSDT-15m-2020-01-05.zip
Downloaded: ETHUSDT-15m-2020-01-06.zip
Unzipped: downloads\ETHUSDT-15m-2020-01-06.zip
Deleted zip file: downloads\ETHUSDT-15m-2020-01-06.zip
Downloaded: E

# 读取数据

In [2]:
import os
import pandas as pd
import glob
from datetime import datetime

# 设置数据文件夹路径
folder_path = './data/BTCUSDT/spot/1d'

# 读取所有CSV文件并合并数据
all_files = glob.glob(os.path.join(folder_path, "BTCUSDT-1d-*.csv"))
df_list = []

for filename in all_files:
    # 读取CSV文件
    temp_df = pd.read_csv(filename, header=None)
    # 设置列名
    temp_df.columns = ['start_time', 'open', 'high', 'low', 'close', 'volume', 'end_time', 'quote_volume', 'trade_count', 'taker_base_vol', 'taker_quote_vol', 'ignore']
    # 提取日期和收盘价格，并将时间戳转换为日期格式
    temp_df['date'] = pd.to_datetime(temp_df['start_time'], unit='ms')
    temp_df = temp_df[['date', 'close']]
    df_list.append(temp_df)

# 合并所有数据到一个DataFrame并按日期排序
btc_df = pd.concat(df_list)
btc_df = btc_df.sort_values(by='date').reset_index(drop=True)

# 查看数据
print(btc_df.head())


        date    close
0 2017-08-17  4285.08
1 2017-08-18  4108.37
2 2017-08-19  4139.98
3 2017-08-20  4086.29
4 2017-08-21  4016.00


# 计算ahr999

## 计算公式
ahr999 = (（比特币价格/200日定投成本) * （比特币价格/指数增长估值）
• 指数增长估值= 10^[5.84 * log(币龄) - 17.01]
• 币龄 = 当前日期距离2009年1月3日的天数

## 数据说明
当比特币价格同时低于短期定投成本和预期估值时增大投资额，能增大用户收益的概率。
根据指标回测，当ahr999指标数据低于0.45时或许适合抄底，在0.45和1.2区间内或许适合定投BTC，高于该区间说明此时或许不是良好的定投时机。

本例采用币安数据，从2017年8月17日开始，初始值参考`coinglass`的数据

In [3]:
import numpy as np
# 计算币龄：当前日期距离2009年1月3日的天数
btc_df['days_since_2009'] = (btc_df['date'] - datetime(2009, 1, 3)).dt.days

# 计算指数增长估值
btc_df['exponential_growth_valuation'] = 10 ** (5.84 * np.log10(btc_df['days_since_2009']) - 17.01)

# 确定2017年8月17日的AHR999和收盘价格，用于反推初始的200日MA
initial_date = datetime(2017, 8, 17)
initial_price = btc_df.loc[btc_df['date'] == initial_date, 'close'].values[0]
initial_valuation = btc_df.loc[btc_df['date'] == initial_date, 'exponential_growth_valuation'].values[0]
initial_ahr999 = 4.2661

# 反推初始的200日定投成本（MA200）
initial_ma200 = (initial_price ** 2) / (initial_ahr999 * initial_valuation)


# 填充200d_ma列
btc_df['200d_ma'] = np.nan

# 将初始200d_ma赋值给2017-08-17，并且对前200天进行累积均值计算
btc_df.loc[btc_df['date'] == initial_date, '200d_ma'] = initial_ma200


# 获取2017-08-17的索引，确保累积计算从该点开始
start_index = btc_df[btc_df['date'] == initial_date].index[0]

# 手动计算前200天的累积均值
for i in range(start_index + 1, len(btc_df)):
    if i < start_index + 200:
        # 手动累积均值计算，除数为200以保持平滑过渡
        btc_df.loc[i, '200d_ma'] = btc_df['200d_ma'].iloc[i - 1] + (btc_df['close'].iloc[i] - btc_df['200d_ma'].iloc[i - 1]) / 200
    else:
        # 超过200天后，使用标准的200日滚动平均
        btc_df.loc[i, '200d_ma'] = btc_df['close'].iloc[i - 199:i + 1].mean()


# 计算AHR999指数
btc_df['ahr999'] = (btc_df['close'] / btc_df['200d_ma']) * (btc_df['close'] / btc_df['exponential_growth_valuation'])

# 查看数据
print(btc_df[['date', 'close', '200d_ma', 'exponential_growth_valuation', 'ahr999']].head(10000))

           date     close       200d_ma  exponential_growth_valuation  \
0    2017-08-17   4285.08   1641.965588                   2621.336570   
1    2017-08-18   4108.37   1654.297610                   2626.203272   
2    2017-08-19   4139.98   1666.726022                   2631.077460   
3    2017-08-20   4086.29   1678.823842                   2635.959143   
4    2017-08-21   4016.00   1690.509723                   2640.848331   
...         ...       ...           ...                           ...   
2630 2024-10-29  72736.42  63299.072550                  90945.417092   
2631 2024-10-30  72344.74  63341.173700                  91037.376894   
2632 2024-10-31  70292.01  63364.324550                  91129.413746   
2633 2024-11-01  69496.01  63394.704650                  91221.527699   
2634 2024-11-02  69374.74  63422.611400                  91313.718804   

        ahr999  
0     4.266100  
1     3.885055  
2     3.908397  
3     3.773241  
4     3.612653  
...        ...  
2630

# 绘图

比特币价格和 AHR999 指数本身在所有数据点上都是完全线性相关的（AHR999 是由 BTC 的价格直接计算得出的）

In [5]:
from pyecharts import options as opts
from pyecharts.charts import Line
import pandas as pd


btc_df['date'] = pd.to_datetime(btc_df['date'])
btc_df.sort_values(by='date', inplace=True)  # 确保数据按日期排序

# # 计算每日增幅百分比
# btc_df['close_pct_change'] = btc_df['close'].pct_change()  # 计算每日比特币价格变化百分比
# btc_df['ahr999_pct_change'] = btc_df['ahr999'].pct_change()  # 计算每日AHR999指数变化百分比

# # 标准化百分比变化列
# btc_df['close_pct_change_normalized'] = (btc_df['close_pct_change'] - btc_df['close_pct_change'].mean()) / btc_df['close_pct_change'].std()
# btc_df['ahr999_pct_change_normalized'] = (btc_df['ahr999_pct_change'] - btc_df['ahr999_pct_change'].mean()) / btc_df['ahr999_pct_change'].std()

# # 使用标准化后的数据计算滚动相关性 (30日)
# rolling_window = 30
# btc_df['rolling_correlation'] = btc_df['close_pct_change_normalized'].rolling(window=rolling_window).corr(btc_df['ahr999_pct_change_normalized'])

# 准备数据
dates = btc_df['date'].dt.strftime("%Y-%m-%d").tolist()
btc_prices = btc_df['close'].tolist()
ahr999_index = btc_df['ahr999'].tolist()
# correlation = btc_df['rolling_correlation'].tolist()

# 创建 Line 图表对象
line = (
    Line(init_opts=opts.InitOpts(bg_color='white'))
    .add_xaxis(dates)
    .add_yaxis(
        "BTC Price (USD)",
        btc_prices,
        yaxis_index=0,
        linestyle_opts=opts.LineStyleOpts(color="blue"),
        label_opts=opts.LabelOpts(is_show=False)
    )
    .add_yaxis(
        "AHR999 Index",
        ahr999_index,
        yaxis_index=1,
        linestyle_opts=opts.LineStyleOpts(color="orange"),
        label_opts=opts.LabelOpts(is_show=False),
        markarea_opts=opts.MarkAreaOpts(  # 使用标注区域来标记 AHR999 的范围
            data=[
                opts.MarkAreaItem(
                    name="DCA Zone",
                    y=(0.45, 1.2),
                    itemstyle_opts=opts.ItemStyleOpts(color="rgba(144, 238, 144, 0.3)")
                )
            ]
        )
    )
    # .add_yaxis(
    #     "30-Day Rolling Correlation",
    #     correlation,
    #     yaxis_index=2,
    #     linestyle_opts=opts.LineStyleOpts(color="green", type_="dashed"),
    #     label_opts=opts.LabelOpts(is_show=False)
    # )
    .extend_axis(
        yaxis=opts.AxisOpts(
            name="AHR999 Index",
            type_="value",
            position="right",
            offset=0,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="orange")
            ),
        )
    )
    # .extend_axis(
    #     yaxis=opts.AxisOpts(
    #         name="30-Day Rolling Correlation",
    #         type_="value",
    #         position="right",
    #         offset=60,  # 偏移量使其与其他轴区分开
    #         axisline_opts=opts.AxisLineOpts(
    #             linestyle_opts=opts.LineStyleOpts(color="green")
    #         ),
    #     )
    # )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="BTC Price, AHR999 Index, and 30-Day Rolling Correlation"),
        tooltip_opts=opts.TooltipOpts(trigger="axis"),
        datazoom_opts=[
            opts.DataZoomOpts(type_="slider", xaxis_index=0),  # 启用时间轴拖动缩放
            opts.DataZoomOpts(type_="inside", xaxis_index=0)   # 启用内嵌缩放
        ],
        yaxis_opts=opts.AxisOpts(
            name="BTC Price (USD)",
            type_="value",
            position="left",
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color="blue")
            ),
        ),
        xaxis_opts=opts.AxisOpts(
            type_="category",
            name="Date",
            boundary_gap=False,
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(width=2)
            ),
        ),
        legend_opts=opts.LegendOpts(pos_top="10%"),
    )
)

# 显示图表
line.render_notebook()
